In [50]:
import re

import requests
import string
from collections import deque
import numpy as np
from itertools import combinations

In [51]:
def import_ChemCad(comps, base_url=None, extract_single_props=None,
                   extract_coeff_props=None, suffix='Props.txt'):

  N_comps = len(comps)
  if base_url is None:
    base_url = 'https://raw.githubusercontent.com/profteachkids/CHE2064/master/data/'

  if extract_single_props is None:
    extract_single_props = {'Molecular Weight' : 'Mw',
                            'Critical Temperature' : 'Tc',
                            'Critical Pressure' : 'Pc',
                            'Critical Volume' : 'Vc',
                            'Acentric factor' : 'w',
                            'Normal boiling point' : 'Tb',
                            'Heat of vaporization' : 'Hvap'}
  if extract_coeff_props is None:
    extract_coeff_props={'Vapor Pressure' : 'Pvap'}

  id_pat = re.compile(r'ID\s+(\d+)')
  formula_pat = re.compile(r'Formula:\s+([A-Z1-9]+)')
  single_props_pat = re.compile('^\s+([\w\s]+?)\s+:\s+([-.0-9e+]+)\s+[\w\s/]*$', re.MULTILINE)
  coeffs_name_pat = re.compile("([\w ]+)\s[^\n]*?Equation.*?Coeffs:([- e\d.+]+)+?", re.DOTALL)
  coeffs_pat = re.compile('([-\de.+]+)')

  props_deque=deque()
  for comp in comps:
    text = requests.get(base_url+comp + suffix).text
    props={'Name': comp}
    props['ID']=id_pat.search(text).groups(1)[0]
    props['Formula']=formula_pat.search(text).groups(1)[0]
    single_props = dict(single_props_pat.findall(text))
    for k,v in extract_single_props.items():
      props[v]=float(single_props.pop(k))

    coeffs_name_strings = dict(coeffs_name_pat.findall(text))
    for k,v in extract_coeff_props.items():
      coeffs = coeffs_pat.findall(coeffs_name_strings[k])
      for letter, value in zip(string.ascii_uppercase,coeffs):
        props[v+letter]=float(value)
    props_deque.append(props)
  props={}
  for prop in props_deque[0].keys():
    if N_comps>1:
        values = np.array([comp[prop] for comp in props_deque])
    else:
        values = props_deque[0][prop]
    props[prop]=values
  return props

In [52]:
def import_NRTL(ids, data_file=None):

    N_comps = len(ids)
    if data_file is None:
     data_file = 'https://raw.githubusercontent.com/profteachkids/CHE2064/master/data/BinaryNRTL.txt'
    text = requests.get(data_file).text

    comps_string = '|'.join(ids)
    id_name_pat = re.compile(r'^\s+(\d+)[ ]+(' + comps_string +')[ ]+[A-Za-z]',re.MULTILINE)
    id_str = id_name_pat.findall(text)
    #maintain order of components
    id_dict = {v:k for k,v in id_str}
    id_str = [id_dict[id] for id in ids]
    comb_strs = combinations(id_str,2)
    comb_indices = combinations(range(N_comps),2)
    NRTL_A, NRTL_B, NRTL_C, NRTL_D, NRTL_alpha = np.zeros((5, N_comps,N_comps))
    for comb_str, comb_index in zip(comb_strs, comb_indices):
        comb_str = '|'.join(comb_str)
        comb_values_pat = re.compile(r'^[ ]+(' + comb_str +
                                     r')[ ]+(?:' + comb_str + r')(.*)$', re.MULTILINE)

        first_id, values = comb_values_pat.search(text).groups(1)
        #if matched order is flipped, also flip indices
        if first_id != comb_index[0]:
            comb_index = (comb_index[1],comb_index[0])
        bij, bji, alpha, aij, aji, cij, cji, dij, dji  = [float(val) for val in values.split()]
        np.add.at(NRTL_B, comb_index, bij)
        np.add.at(NRTL_B, (comb_index[1],comb_index[0]), bji)
        np.add.at(NRTL_A, comb_index, aij)
        np.add.at(NRTL_A, (comb_index[1],comb_index[0]), aji)
        np.add.at(NRTL_C, comb_index, cij)
        np.add.at(NRTL_C, (comb_index[1],comb_index[0]), cji)
        np.add.at(NRTL_D, comb_index, dij)
        np.add.at(NRTL_D, (comb_index[1],comb_index[0]), dji)
        np.add.at(NRTL_alpha, comb_index, alpha)
        np.add.at(NRTL_alpha, (comb_index[1],comb_index[0]), alpha)
    return dict(A=NRTL_A, B=NRTL_B, C=NRTL_C, D=NRTL_D, alpha=NRTL_alpha)

In [55]:
props=import_ChemCad(['Propylene','Isopropanol','Water'])
import_NRTL(props['ID'])

AttributeError: 'NoneType' object has no attribute 'groups'

In [54]:
props

{'Name': array(['Isopropanol', 'Water', 'Toluene'], dtype='<U11'),
 'ID': array(['145', '62', '41'], dtype='<U3'),
 'Formula': array(['C3H8O', 'H2O', 'C7H8'], dtype='<U5'),
 'Mw': array([60.096, 18.015, 92.141]),
 'Tc': array([508.31, 647.35, 591.79]),
 'Pc': array([ 4764300., 22118230.,  4108000.]),
 'Vc': array([0.2201  , 0.063494, 0.316   ]),
 'w': array([0.6689, 0.348 , 0.264 ]),
 'Tb': array([355.41, 373.15, 383.78]),
 'Hvap': array([39832530., 40656800., 33179830.]),
 'PvapA': array([92.935, 72.55 , 76.945]),
 'PvapB': array([-8177.1, -7206.7, -6729.8]),
 'PvapC': array([-10.031 ,  -7.1385,  -8.179 ]),
 'PvapD': array([3.9988e-06, 4.0460e-06, 5.3017e-06]),
 'PvapE': array([2., 2., 2.]),
 'PvapF': array([0., 0., 0.]),
 'PvapG': array([0., 0., 0.])}